In [188]:
%run 'Data Cleaning.ipynb'

In [89]:
jobs_cleaned.head()

,job_id,company_id,title,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,...,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,scraped,skill_name,industry_name
0,3757940104,553718.0,Hearing Care Provider,Overview\n\nHearingLife is a national hearing ...,NaN,63000.0,NaN,MONTHLY,Full-time,"Little River, SC",...,NaN,1.699090e+12,careers-demant.icims.com,0,FULL_TIME,USD,BASE_SALARY,1699138101,Other,Medical Equipment Manufacturing
4,3757938018,18213359.0,Cook,descriptionTitle\n\n Looking for a great oppor...,NaN,46321.6,NaN,HOURLY,Full-time,"Aliso Viejo, CA",...,NaN,1.699080e+12,jobs.apploi.com,0,FULL_TIME,USD,BASE_SALARY,1699087461,Management,Non-profit Organizations
5,3757938018,18213359.0,Cook,descriptionTitle\n\n Looking for a great oppor...,NaN,46321.6,NaN,HOURLY,Full-time,"Aliso Viejo, CA",...,NaN,1.699080e+12,jobs.apploi.com,0,FULL_TIME,USD,BASE_SALARY,1699087461,Manufacturing,Non-profit Organizations
6,3757937095,437225.0,Principal Cloud Security Architect (Remote),"Job Summary\nAt iHerb, we are on a mission to ...",275834.0,240895.0,205956.0,YEARLY,Full-time,United States,...,NaN,1.699090e+12,careers.iherb.com,0,FULL_TIME,USD,BASE_SALARY,1699085346,Information Technology,"Technology, Information and Internet"
7,3757937095,437225.0,Principal Cloud Security Architect (Remote),"Job Summary\nAt iHerb, we are on a mission to ...",275834.0,240895.0,205956.0,YEARLY,Full-time,United States,...,NaN,1.699090e+12,careers.iherb.com,0,FULL_TIME,USD,BASE_SALARY,1699085346,Information Technology,Manufacturing


In [177]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

med_P = np.percentile(jobs_cleaned['med_salary'], [1, 99])
jobs_cleaned = jobs_cleaned[(jobs_cleaned['med_salary'] > med_P[0]) & (jobs_cleaned['med_salary'] < med_P[1])]

min_P = np.percentile(jobs_cleaned['min_salary'], [1, 99])
jobs_cleaned = jobs_cleaned[(jobs_cleaned['min_salary'] > min_P[0]) & (jobs_cleaned['min_salary'] < min_P[1])]

max_P = np.percentile(jobs_cleaned['max_salary'], [1, 99])
jobs_cleaned = jobs_cleaned[(jobs_cleaned['min_salary'] > max_P[0]) & (jobs_cleaned['min_salary'] < max_P[1])]

# Create a new LabelEncoder for each categorical column
title_encoder = LabelEncoder()
location_encoder = LabelEncoder()
skill_name_encoder = LabelEncoder()
experience_level_encoder = LabelEncoder()

# Fit and transform each column with its respective encoder
jobs_cleaned['title_enc'] = title_encoder.fit_transform(jobs_cleaned['title'])
jobs_cleaned['location_enc'] = location_encoder.fit_transform(jobs_cleaned['location'])
jobs_cleaned['skill_name_enc'] = skill_name_encoder.fit_transform(jobs_cleaned['skill_name'])
jobs_cleaned['formatted_experience_level_enc'] = experience_level_encoder.fit_transform(jobs_cleaned['formatted_experience_level'])


X = jobs_cleaned[['title_enc', 'location_enc','skill_name_enc','formatted_experience_level_enc']]


In [183]:
def train_salary_model(salary_level):
    # Select the salary level for prediction
    y = jobs_cleaned[salary_level]
    
    # Splitting the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initializing and training the XGBoost regressor model
    xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', learning_rate = 0.2,
                              max_depth = 5, n_estimators = 1000)
    xg_reg.fit(X_train, y_train)

    # Making predictions on the test set
    y_pred = xg_reg.predict(X_test)

    # Calculating the RMSE for the predictions
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    # Printing the RMSE
#     print(f"{salary_level} RMSE: {rmse}")
    
    # Returning the trained model
    return xg_reg

In [181]:
train_salary('max_salary')

max_salary RMSE: 8388.852965490343


# XGBoost

### Median Salary

In [153]:
# med_X_train, med_X_test, med_y_train, med_y_test = train_test_split(X, y_median, test_size=0.2, random_state=42)

# med_xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', learning_rate = 0.2,
#                 max_depth = 5, n_estimators = 1000)

# med_xg_reg.fit(med_X_train, med_y_train)

# med_y_pred = med_xg_reg.predict(med_X_test)

# median_rmse = mean_squared_error(med_y_test, med_y_pred, squared=False)
# print(f"Median RMSE: {median_rmse}")

Median RMSE: 8913.971176292791


### Max Salary

In [154]:
# import xgboost as xgb
# from sklearn.metrics import mean_squared_error

# max_X_train, max_X_test, max_y_train, max_y_test = train_test_split(X, y_max, test_size=0.2, random_state=42)

# max_xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', learning_rate = 0.2,
#                 max_depth = 5, n_estimators = 1000)

# max_xg_reg.fit(max_X_train, max_y_train)

# max_y_pred = max_xg_reg.predict(max_X_test)

# max_rmse = mean_squared_error(max_y_test, max_y_pred, squared=False)
# print(f"Max RMSE: {max_rmse}")

Max RMSE: 11548.796822810427


### Min Salary

In [167]:
# import xgboost as xgb
# from sklearn.metrics import mean_squared_error

# min_X_train, min_X_test, min_y_train, min_y_test = train_test_split(X, y_min, test_size=0.2, random_state=42)

# min_xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', learning_rate = 0.2,
#                 max_depth = 5, n_estimators = 1000)

# min_xg_reg.fit(min_X_train, min_y_train)

# min_y_pred = min_xg_reg.predict(min_X_test)

# min_rmse = mean_squared_error(min_y_test, min_y_pred, squared=False)
# print(f"Minimum RMSE: {min_rmse}")

Minimum RMSE: 6302.6979595513485


### Min, Median, Max estimator

In [185]:
med_salary_model = train_salary_model('med_salary')
min_salary_model = train_salary_model('min_salary')
max_salary_model = train_salary_model('max_salary')

# Example user input
for location in jobs_cleaned['location'].unique():
    user_input = {
        'title_enc': ['Software Engineer'],
        'location_enc': [f'{location}'],
        'skill_name_enc': ['Information Technology'],
        'formatted_experience_level_enc': ['Mid-Senior level']
    }

    user_input_df = pd.DataFrame(user_input)

    user_input_df['title_enc'] = title_encoder.transform(user_input_df['title_enc'])
    user_input_df['location_enc'] = location_encoder.transform(user_input_df['location_enc'])
    user_input_df['skill_name_enc'] = skill_name_encoder.transform(user_input_df['skill_name_enc'])
    user_input_df['formatted_experience_level_enc'] = experience_level_encoder.transform(user_input_df['formatted_experience_level_enc'])

    # Predict the salary for the user input
    min_predicted_salary = min_salary_model.predict(user_input_df)
    median_predicted_salary = med_salary_model.predict(user_input_df)
    max_predicted_salary = max_salary_model.predict(user_input_df)

    print(f"----{location}-----")
    print(f"Minimum Estimated salary: {min_predicted_salary[0]:.2f} USD/year")
    print(f"Median Estimated salary: {median_predicted_salary[0]:.2f} USD/year")
    print(f"Maximum Estimated salary: {max_predicted_salary[0]:.2f} USD/year")
    

----Broomfield, CO-----
Minimum Estimated salary: 86891.91 USD/year
Median Estimated salary: 104579.80 USD/year
Maximum Estimated salary: 124084.39 USD/year
----Swedesboro, NJ-----
Minimum Estimated salary: 99247.23 USD/year
Median Estimated salary: 108882.02 USD/year
Maximum Estimated salary: 124016.81 USD/year
----Grand Rapids metropolitan area, MI-----
Minimum Estimated salary: 94713.66 USD/year
Median Estimated salary: 106201.76 USD/year
Maximum Estimated salary: 109895.85 USD/year
----Dunn, NC-----
Minimum Estimated salary: 91384.75 USD/year
Median Estimated salary: 105572.95 USD/year
Maximum Estimated salary: 128252.09 USD/year
----Encinitas, CA-----
Minimum Estimated salary: 88788.80 USD/year
Median Estimated salary: 101626.94 USD/year
Maximum Estimated salary: 115147.94 USD/year
----Oceanside, CA-----
Minimum Estimated salary: 88276.91 USD/year
Median Estimated salary: 101839.14 USD/year
Maximum Estimated salary: 116275.95 USD/year
----United States-----
Minimum Estimated salar

----Grand Rapids, MI-----
Minimum Estimated salary: 90411.87 USD/year
Median Estimated salary: 93887.73 USD/year
Maximum Estimated salary: 102222.70 USD/year
----Boulder, CO-----
Minimum Estimated salary: 85051.64 USD/year
Median Estimated salary: 96357.48 USD/year
Maximum Estimated salary: 112195.05 USD/year
----Muskegon, MI-----
Minimum Estimated salary: 93418.10 USD/year
Median Estimated salary: 117651.93 USD/year
Maximum Estimated salary: 140466.45 USD/year
----Greater Phoenix Area-----
Minimum Estimated salary: 94062.17 USD/year
Median Estimated salary: 106819.02 USD/year
Maximum Estimated salary: 120809.20 USD/year
----Orange County, CA-----
Minimum Estimated salary: 96992.73 USD/year
Median Estimated salary: 114109.91 USD/year
Maximum Estimated salary: 131565.80 USD/year
----Dublin, CA-----
Minimum Estimated salary: 90410.18 USD/year
Median Estimated salary: 106098.84 USD/year
Maximum Estimated salary: 128945.57 USD/year
----Montana, United States-----
Minimum Estimated salary: 

----Vancouver, WA-----
Minimum Estimated salary: 104691.42 USD/year
Median Estimated salary: 119947.95 USD/year
Maximum Estimated salary: 128866.12 USD/year
----Dayton, OH-----
Minimum Estimated salary: 87769.05 USD/year
Median Estimated salary: 102106.80 USD/year
Maximum Estimated salary: 123585.66 USD/year
----Greater New Orleans Region-----
Minimum Estimated salary: 93608.11 USD/year
Median Estimated salary: 104716.24 USD/year
Maximum Estimated salary: 116509.43 USD/year
----Boise, ID-----
Minimum Estimated salary: 81792.98 USD/year
Median Estimated salary: 98494.46 USD/year
Maximum Estimated salary: 123997.69 USD/year
----Utah, United States-----
Minimum Estimated salary: 104691.42 USD/year
Median Estimated salary: 119947.95 USD/year
Maximum Estimated salary: 128866.12 USD/year
----Jefferson County, IL-----
Minimum Estimated salary: 89509.23 USD/year
Median Estimated salary: 103785.48 USD/year
Maximum Estimated salary: 119311.93 USD/year
----Rocky Hill, CT-----
Minimum Estimated sa

----Tipton, IN-----
Minimum Estimated salary: 108518.57 USD/year
Median Estimated salary: 113537.73 USD/year
Maximum Estimated salary: 117621.70 USD/year
----Carrollton, GA-----
Minimum Estimated salary: 86950.36 USD/year
Median Estimated salary: 102444.35 USD/year
Maximum Estimated salary: 120720.23 USD/year
----Riverside, CA-----
Minimum Estimated salary: 101175.21 USD/year
Median Estimated salary: 109508.96 USD/year
Maximum Estimated salary: 126657.30 USD/year
----Irvine, CA-----
Minimum Estimated salary: 94512.81 USD/year
Median Estimated salary: 107806.90 USD/year
Maximum Estimated salary: 126115.66 USD/year
----Greater Birmingham, Alabama Area-----
Minimum Estimated salary: 95055.70 USD/year
Median Estimated salary: 111855.50 USD/year
Maximum Estimated salary: 127938.40 USD/year
----Greater Seattle Area-----
Minimum Estimated salary: 96233.91 USD/year
Median Estimated salary: 107008.90 USD/year
Maximum Estimated salary: 118745.36 USD/year
----Glenview, IL-----
Minimum Estimated s

----Edwards, CA-----
Minimum Estimated salary: 92042.61 USD/year
Median Estimated salary: 107424.10 USD/year
Maximum Estimated salary: 120854.23 USD/year
----Falconer, NY-----
Minimum Estimated salary: 88233.69 USD/year
Median Estimated salary: 100673.58 USD/year
Maximum Estimated salary: 114792.66 USD/year
----Oklahoma City, OK-----
Minimum Estimated salary: 94376.62 USD/year
Median Estimated salary: 101839.14 USD/year
Maximum Estimated salary: 117342.23 USD/year
----Little Rock, AR-----
Minimum Estimated salary: 87727.40 USD/year
Median Estimated salary: 101230.60 USD/year
Maximum Estimated salary: 116380.54 USD/year
----Cupertino, CA-----
Minimum Estimated salary: 97611.95 USD/year
Median Estimated salary: 106441.34 USD/year
Maximum Estimated salary: 118985.88 USD/year
----Decorah, IA-----
Minimum Estimated salary: 89398.06 USD/year
Median Estimated salary: 100101.23 USD/year
Maximum Estimated salary: 126324.03 USD/year
----Shoreline, WA-----
Minimum Estimated salary: 102268.88 USD/

----Jersey City, NJ-----
Minimum Estimated salary: 89573.84 USD/year
Median Estimated salary: 104930.85 USD/year
Maximum Estimated salary: 119858.61 USD/year
----Blue Bell, PA-----
Minimum Estimated salary: 83608.70 USD/year
Median Estimated salary: 101727.95 USD/year
Maximum Estimated salary: 123767.04 USD/year
----Eugene, OR-----
Minimum Estimated salary: 86059.24 USD/year
Median Estimated salary: 99988.66 USD/year
Maximum Estimated salary: 114576.16 USD/year
----Red Oak, TX-----
Minimum Estimated salary: 96837.40 USD/year
Median Estimated salary: 111001.81 USD/year
Maximum Estimated salary: 120902.99 USD/year
----Pueblo-Cañon City Area-----
Minimum Estimated salary: 99520.41 USD/year
Median Estimated salary: 107887.84 USD/year
Maximum Estimated salary: 120085.10 USD/year
----Iowa, United States-----
Minimum Estimated salary: 93754.27 USD/year
Median Estimated salary: 109883.62 USD/year
Maximum Estimated salary: 134293.42 USD/year
----Greater Macon-----
Minimum Estimated salary: 9309

----Kohler, WI-----
Minimum Estimated salary: 95911.59 USD/year
Median Estimated salary: 107170.63 USD/year
Maximum Estimated salary: 122518.69 USD/year
----Lowell, AR-----
Minimum Estimated salary: 93717.03 USD/year
Median Estimated salary: 110903.80 USD/year
Maximum Estimated salary: 125696.88 USD/year
----Syracuse, NY-----
Minimum Estimated salary: 98955.35 USD/year
Median Estimated salary: 108882.02 USD/year
Maximum Estimated salary: 125006.62 USD/year
----Lebanon, NH-----
Minimum Estimated salary: 89439.01 USD/year
Median Estimated salary: 102803.57 USD/year
Maximum Estimated salary: 116658.76 USD/year
----Lexington, KY-----
Minimum Estimated salary: 86849.24 USD/year
Median Estimated salary: 102888.74 USD/year
Maximum Estimated salary: 119032.13 USD/year
----Maryville, TN-----
Minimum Estimated salary: 89324.31 USD/year
Median Estimated salary: 103563.45 USD/year
Maximum Estimated salary: 112774.19 USD/year
----Chattanooga, TN-----
Minimum Estimated salary: 83216.80 USD/year
Medi

----Kirkland, WA-----
Minimum Estimated salary: 96422.98 USD/year
Median Estimated salary: 109660.31 USD/year
Maximum Estimated salary: 124798.19 USD/year
----Bedford Hills, NY-----
Minimum Estimated salary: 85433.28 USD/year
Median Estimated salary: 106374.26 USD/year
Maximum Estimated salary: 132802.45 USD/year
----Santa Clarita, California, United States-----
Minimum Estimated salary: 96695.12 USD/year
Median Estimated salary: 110177.62 USD/year
Maximum Estimated salary: 125502.04 USD/year
----McCamey, TX-----
Minimum Estimated salary: 89383.73 USD/year
Median Estimated salary: 103351.68 USD/year
Maximum Estimated salary: 114452.54 USD/year
----San Dimas, CA-----
Minimum Estimated salary: 95464.63 USD/year
Median Estimated salary: 105448.64 USD/year
Maximum Estimated salary: 119186.25 USD/year
----Brea, CA-----
Minimum Estimated salary: 86339.18 USD/year
Median Estimated salary: 99523.92 USD/year
Maximum Estimated salary: 112773.38 USD/year
----Lenoir, NC-----
Minimum Estimated sala

In [98]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'max_depth': [3, 4, 5],
    'n_estimators': [50, 100, 200, 500, 1000],
    'learning_rate': [0.01, 0.1, 0.2]
}

xg_reg = xgb.XGBRegressor(objective ='reg:squarederror')

grid_search = GridSearchCV(estimator=xg_reg, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')

grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)

Best parameters: {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 1000}


In [99]:
y_pred = grid_search.predict(X_test)

In [100]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE: {rmse}")

RMSE: 22558.93944458922


# Catboost Regressor

In [15]:
from feature_engine.encoding import RareLabelEncoder

# select label
df = jobs_cleaned[['title','location','skill_name','med_salary','formatted_experience_level','industry_name']]
main_label = 'med_salary'
# Exclude 1% of smallest and 1% of highest values
P = np.percentile(df[main_label], [1, 99])
df = df[(df[main_label] > P[0]) & (df[main_label] < P[1])]

# log10-transform columns and group by larger bins
def log10_transform(x):
    try: 
        return str(round(1/5*round(5*np.log10(1+x)),1))
    except:
        return 'None'
# set up the rare label encoder limiting number of categories to max_n_categories
for col in ['title','location','skill_name','formatted_experience_level','industry_name']:
    df[col] = df[col].fillna('None')
    encoder = RareLabelEncoder(n_categories=1, max_n_categories=50, replace_with='Other', tol=10.0/df.shape[0])
    df[col] = encoder.fit_transform(df[[col]])
print(df.shape)
df.sample(5).T

(27776, 6)


,67996,18468,43507,63213,73671
title,Other,Other,Sales Director [Owner/Operator],Retail Sales Associate,Other
location,"Miami, FL","Raleigh, NC",Other,Other,Other
skill_name,Management,Finance,Business Development,Other,Analyst
med_salary,58000.0,107900.0,201182.0,49500.0,60000.0
formatted_experience_level,Mid-Senior level,None,Director,Entry level,Mid-Senior level
industry_name,Hospitality,Software Development,Book and Periodical Publishing,Telecommunications,Food and Beverage Manufacturing


In [16]:
# initialize data
y = df[main_label].values.reshape(-1,)
X = df[['title','location','skill_name','formatted_experience_level','industry_name']]
cat_cols = df.select_dtypes(include=['object']).columns
cat_cols_idx = [list(X.columns).index(c) for c in cat_cols]
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.5, random_state=0, stratify=df[['title']])
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((13888, 5), (13888, 5), (13888,), (13888,))

In [17]:
from catboost import Pool, CatBoostRegressor

# initialize Pool
train_pool = Pool(X_train, 
                  y_train, 
                  cat_features=cat_cols_idx)
test_pool = Pool(X_test,
                 y_test,
                 cat_features=cat_cols_idx)
# specify the training parameters 
model = CatBoostRegressor(iterations=2000, 
                          depth=5,
                          verbose=0,
                          learning_rate=0.01, 
                          loss_function='RMSE')
#train the model
model.fit(train_pool)
# make the prediction using the resulting model
y_train_pred = model.predict(train_pool)
y_test_pred = model.predict(test_pool)

rmse_train = mean_squared_error(y_train, y_train_pred, squared=False)
rmse_test = mean_squared_error(y_test, y_test_pred, squared=False)
print(f"RMSE score for train {round(rmse_train,3)} USD/year, and for test {round(rmse_test,3)} USD/year")

RMSE score for train 36052.732 USD/year, and for test 37479.148 USD/year


In [18]:
# Example user input
for location in jobs_cleaned['location'].unique():
    user_input = {
        'title': ['Data Scientist'],
        'location': [f'{location}'],
        'skill_name': ['Information Technology'],
        'formatted_experience_level': ['Mid-Senior level'],
        'industry_name': ['Technology, Information and Internet']
    }

    user_input_df = pd.DataFrame(user_input)

    # Predict the salary for the user input
    predicted_salary = model.predict(user_input_df)
    print(f"----{location}-----")
    print(f"Estimated salary: {predicted_salary[0]:.2f} USD/year")


----Little River, SC-----
Estimated salary: 124162.41 USD/year
----Aliso Viejo, CA-----
Estimated salary: 124162.41 USD/year
----United States-----
Estimated salary: 116082.74 USD/year
----Coeur d'Alene, ID-----
Estimated salary: 124162.41 USD/year
----Altadena, CA-----
Estimated salary: 124162.41 USD/year
----Waukesha, WI-----
Estimated salary: 124162.41 USD/year
----Orlando, FL-----
Estimated salary: 115969.25 USD/year
----Oakland, CA-----
Estimated salary: 124976.64 USD/year
----Broomfield, CO-----
Estimated salary: 124162.41 USD/year
----Swedesboro, NJ-----
Estimated salary: 124162.41 USD/year
----Richmond, VA-----
Estimated salary: 124162.41 USD/year
----Rapid City, SD-----
Estimated salary: 124162.41 USD/year
----Yuma, AZ-----
Estimated salary: 124162.41 USD/year
----Sheboygan, WI-----
Estimated salary: 124162.41 USD/year
----Portland, OR-----
Estimated salary: 111863.21 USD/year
----Urbana-Champaign Area-----
Estimated salary: 124162.41 USD/year
----Marietta, GA-----
Estimated s

----Woodinville, WA-----
Estimated salary: 124162.41 USD/year
----Cape May Court House, NJ-----
Estimated salary: 124162.41 USD/year
----Greenwich, CT-----
Estimated salary: 124162.41 USD/year
----Pascagoula, MS-----
Estimated salary: 124162.41 USD/year
----Waldwick, NJ-----
Estimated salary: 124162.41 USD/year
----Wayne, PA-----
Estimated salary: 124162.41 USD/year
----Groveland, CA-----
Estimated salary: 124162.41 USD/year
----Vermont, United States-----
Estimated salary: 124162.41 USD/year
----Loma Linda, CA-----
Estimated salary: 124162.41 USD/year
----Nashua, NH-----
Estimated salary: 124162.41 USD/year
----Sarasota, FL-----
Estimated salary: 124162.41 USD/year
----Waterville, ME-----
Estimated salary: 124162.41 USD/year
----Livermore, CA-----
Estimated salary: 124162.41 USD/year
----Laguna Beach, CA-----
Estimated salary: 124162.41 USD/year
----Osage Beach, MO-----
Estimated salary: 124162.41 USD/year
----Carlsbad, CA-----
Estimated salary: 124162.41 USD/year
----Columbus, Ohio, 

----Spartanburg, SC-----
Estimated salary: 124162.41 USD/year
----Bellingham, WA-----
Estimated salary: 124162.41 USD/year
----Rochester, NH-----
Estimated salary: 124162.41 USD/year
----North Salem, NY-----
Estimated salary: 124162.41 USD/year
----Eunice, NM-----
Estimated salary: 124162.41 USD/year
----Westlake Village, CA-----
Estimated salary: 124162.41 USD/year
----Thousand Oaks, CA-----
Estimated salary: 124162.41 USD/year
----Sanford, NC-----
Estimated salary: 124162.41 USD/year
----Jeffersonville, OH-----
Estimated salary: 124162.41 USD/year
----Batavia, IL-----
Estimated salary: 124162.41 USD/year
----Syosset, NY-----
Estimated salary: 124162.41 USD/year
----Burnsville, MN-----
Estimated salary: 124162.41 USD/year
----Elyria, OH-----
Estimated salary: 124162.41 USD/year
----Oak Brook, IL-----
Estimated salary: 124162.41 USD/year
----Corte Madera, CA-----
Estimated salary: 124162.41 USD/year
----Lynwood, CA-----
Estimated salary: 124162.41 USD/year
----Duarte, CA-----
Estimated

----West Long Branch, NJ-----
Estimated salary: 124162.41 USD/year
----Athens, OH-----
Estimated salary: 124162.41 USD/year
----Hilton Head Island, South Carolina Area-----
Estimated salary: 124162.41 USD/year
----Arkadelphia, AR-----
Estimated salary: 124162.41 USD/year
----Red Bank, NJ-----
Estimated salary: 124162.41 USD/year
----Burlington, WA-----
Estimated salary: 124162.41 USD/year
----West Chicago, IL-----
Estimated salary: 124162.41 USD/year
----St Helena, CA-----
Estimated salary: 124162.41 USD/year
----Sanford, FL-----
Estimated salary: 124162.41 USD/year
----Braselton, GA-----
Estimated salary: 124162.41 USD/year
----Hamilton County, OH-----
Estimated salary: 124162.41 USD/year
----Portland, TN-----
Estimated salary: 124162.41 USD/year
----Hermitage, PA-----
Estimated salary: 124162.41 USD/year
----Great Falls, MT-----
Estimated salary: 124162.41 USD/year
----Burlington, VT-----
Estimated salary: 124162.41 USD/year
----Batavia, OH-----
Estimated salary: 124162.41 USD/year
-